# Convert playground data to DB seed files

The playground data is retrieved from various sources in varying formats. Convert it to a format that can be used to insert the data into the database.

In [42]:
import geopandas as gpd
from pathlib import Path
import osmnx

In [43]:
repo_root = Path().resolve().parent
data_dir = repo_root / "data"
playgrounds_dir = data_dir / "playgrounds"
seed_dir = repo_root / "supabase" / "seeds"

In [44]:
cities_with_open_data = ['Helsinki', 'Espoo', 'Vantaa', 'Kauniainen', 'Turku', 'Tampere', 'Oulu']

In [45]:
def write_seed_file(playground_data: gpd.GeoDataFrame, output_path: Path, crs: int):
    header = """
-- Import playgrounds
INSERT INTO playgrounds (name, address, location, description, has_supervised_activities, city, data_source)
VALUES
"""

    values = []
    for _, row in playground_data.iterrows():
        # Get coordinates from the geometry
        x, y = row.geometry.x, row.geometry.y
        description = None
        name = row["name"]
        address = row.get("address")
        city = row.get("city")
        has_supervised = bool(row.get("has_supervised_activities", False))
        data_source = row.get("data_source")

        # Escape single quotes by doubling them
        name = name.replace("'", "''") if name is not None else None
        address = address.replace("'", "''") if address is not None else None
        city = city.replace("'", "''") if city is not None else None
        description = description.replace("'", "''") if description is not None else None

        value = f"""    (
        {"'" + name + "'" if name is not None else "NULL"},
        {"'" + address + "'" if address is not None else "NULL"},
        ST_Transform(ST_SetSRID(ST_MakePoint({x}, {y}), {crs}), 4326),
        {"'" + description + "'" if description is not None else "NULL"},
        {str(has_supervised).lower()},
        {"'" + city + "'" if city is not None else "NULL"},
        {"'" + data_source + "'"}
    )"""
        values.append(value)


    with open(output_path, "w") as f:
        f.write(header)
        f.write(",\n".join(values))
        f.write(";\n")

# Helsinki

Data for Helsinki can be retrieved from multiple sources. Likely the easiest is to go to https://kartta.hel.fi/ and search for Leikkipuistot. From there you can download the data in multiple formats, Geopackage among others.

In [46]:
helsinki_input_path = playgrounds_dir / "helsinki.gpkg"
helsinki_seed_path = seed_dir / "seed_helsinki.sql"
column_mapping = {
    "Name_fi": "name",
    "Address_fi": "address",
    "Selite_fi": "selite",
    "geometry": "geometry"
}

helsinki_playgrounds = gpd.read_file(helsinki_input_path)
helsinki_playgrounds = helsinki_playgrounds.rename(columns=column_mapping)
helsinki_playgrounds = helsinki_playgrounds[column_mapping.values()]

helsinki_playgrounds["city"] = None

helsinki_playgrounds.loc[helsinki_playgrounds["address"].str.contains("Helsinki", na=False), "city"] = "Helsinki"
helsinki_playgrounds.loc[helsinki_playgrounds["address"].str.contains("Espoo", na=False), "city"] = "Espoo"
helsinki_playgrounds.loc[helsinki_playgrounds["address"].str.contains("Vantaa", na=False), "city"] = "Vantaa"
helsinki_playgrounds.loc[helsinki_playgrounds["address"].str.contains("Kauniainen", na=False), "city"] = "Kauniainen"

helsinki_playgrounds["has_supervised_activities"] = False
helsinki_playgrounds.loc[helsinki_playgrounds["selite"].str.contains("Ohjattu leikkipuistotoiminta", na=False), "has_supervised_activities"] = True

helsinki_playgrounds["data_source"] = "municipality"
helsinki_crs = helsinki_playgrounds.geometry.crs.to_epsg() # should be epsg:3879

write_seed_file(helsinki_playgrounds, helsinki_seed_path, helsinki_crs)

helsinki_playgrounds.head()

,name,address,selite,geometry,city,has_supervised_activities,data_source
0,Olarin asukaspuisto,"Meteorinkatu 7, 02210 Espoo",Ohjattu leikkipuistotoiminta,POINT (25485095 6672856),Espoo,True,municipality
1,Leikkipuisto Hilleri,"Hillerikuja 6, 00800 Helsinki",Ohjattu leikkipuistotoiminta,POINT (25502459 6677431),Helsinki,True,municipality
2,Kivenlahden asukaspuisto,"Merivirta 12 A, 02320 Espoo",Ohjattu leikkipuistotoiminta,POINT (25479780 6671150),Espoo,True,municipality
3,Leikkipuisto Vähätupa,"Vähäntuvantie 4a, 00390 Helsinki",Ohjattu leikkipuistotoiminta,POINT (25491857 6680089),Helsinki,True,municipality
4,Hiirisuon asukaspuisto,"Pohjoisentie 1, 02970 Espoo",Ohjattu leikkipuistotoiminta,POINT (25485540 6687811),Espoo,True,municipality


# Turku

Data for Turku can be retrieved from https://www.avoindata.fi/data/fi/dataset/turun-ja-kaarinan-leikkipaikat. Best data seems to be in the OGC API from which I downloaded the data to a Geopackage file using QGIS.

In [47]:
turku_input_path = playgrounds_dir / "turku.gpkg"
turku_seed_path = seed_dir / "seed_turku.sql"
column_mapping = {
    "Omistaja": "city",
    "Leikkipaikkanimi": "name",
    "geometry": "geometry"
}

turku_playgrounds = gpd.read_file(turku_input_path)
turku_playgrounds = turku_playgrounds.rename(columns=column_mapping)
turku_playgrounds = turku_playgrounds[column_mapping.values()]

city_mapping = {'Turun kaupunki': 'Turku', 'Kaarinan kaupunki': 'Kaarina', 'Ei tietoa': None}
turku_playgrounds["city"] = turku_playgrounds["city"].map(city_mapping)

# there is a playground with name "Tenderipuiston leikkipaikka" that does not have city set
turku_playgrounds.loc[turku_playgrounds["name"] == "Tenderipuiston leikkipaikka", "city"] = "Turku"

turku_playgrounds["has_supervised_activities"] = None
turku_playgrounds["address"] = None

turku_playgrounds["data_source"] = "municipality"
turku_crs = turku_playgrounds.geometry.crs.to_epsg() # should be epsg:3877

write_seed_file(turku_playgrounds, turku_seed_path, turku_crs)

turku_playgrounds.head()

,city,name,geometry,has_supervised_activities,address,data_source
0,Turku,Hemmilän leikkipaikka,POINT Z (23455754.67 6705024.592 0),None,None,municipality
1,Turku,Piiparinpuiston leikkipaikka,POINT Z (23459124.497 6708537.691 0),None,None,municipality
2,Turku,Hyppäskentän leikkipaikka,POINT Z (23461782.209 6714961.993 0),None,None,municipality
3,Turku,Tyytikinpesän leikkipaikka,POINT Z (23461463.973 6715104.899 0),None,None,municipality
4,Turku,Iskoisten leikkipaikka,POINT Z (23466072.226 6720572.732 0),None,None,municipality


# Tampere

Data for Tampere can be retrieved from https://data.tampere.fi/data/fi/dataset/tampereen-leikkipaikat.

In [48]:
tampere_input_path = playgrounds_dir / "tampere.json"
tampere_seed_path = seed_dir / "seed_tampere.sql"
column_mapping = {
    "viheralueen_nimi_tunnus": "name",
    "geometry": "geometry"
}

tampere_playgrounds = gpd.read_file(tampere_input_path)
tampere_playgrounds = tampere_playgrounds.rename(columns=column_mapping)
tampere_playgrounds = tampere_playgrounds[column_mapping.values()]

tampere_playgrounds["city"] = "Tampere"
tampere_playgrounds["has_supervised_activities"] = None
tampere_playgrounds["address"] = None

# names are all messed up
# 837IHAISTENPUISTO 2582-005 -> Ihaistenpuisto
# 837NURMIN LEIKKIPAIKKA 0043-001 -> Nurmin leikkipaikka
# drop first three characters from name
tampere_playgrounds["name"] = tampere_playgrounds["name"].str[3:]
# drop everything after last space
tampere_playgrounds["name"] = tampere_playgrounds["name"].str.rsplit(" ", n=1).str[0]
# Capitalize first letter of name
tampere_playgrounds["name"] = tampere_playgrounds["name"].str.capitalize()

tampere_playgrounds["data_source"] = "municipality"
tampere_crs = tampere_playgrounds.geometry.crs.to_epsg() # should be epsg:3878

write_seed_file(tampere_playgrounds, tampere_seed_path, tampere_crs)

tampere_playgrounds.head()

,name,geometry,city,has_supervised_activities,address,data_source
0,Ihaistenpuisto,POINT (24482524.156 6825507.527),Tampere,None,None,municipality
1,Haavistonpuisto,POINT (24491120.78 6835333.097),Tampere,None,None,municipality
2,Kämmensuonpuisto,POINT (24490671.682 6835407.566),Tampere,None,None,municipality
3,Nurmin leikkipaikka,POINT (24496769.219 6825692.551),Tampere,None,None,municipality
4,Terälahden leikkipaikka,POINT (24494697.503 6844377.582),Tampere,None,None,municipality


# Oulu

Data for Oulu can be retrieved from https://data.ouka.fi/data/fi/dataset/kalastusalueet/resource/13fe557b-8265-4303-8b6b-037058a6a083 which points to a WFS service https://e-kartta.ouka.fi/TeklaOgcWebOpen/WFS.ashx which has a layer gis:leikkipaikat.

In [49]:
oulu_input_path = playgrounds_dir / "oulu.gpkg"
oulu_seed_path = seed_dir / "seed_oulu.sql"
column_mapping = {
    "Tunnus": "name",
    "geometry": "geometry"
}

oulu_playgrounds = gpd.read_file(oulu_input_path)
oulu_playgrounds = oulu_playgrounds.rename(columns=column_mapping)
oulu_playgrounds = oulu_playgrounds[column_mapping.values()]

oulu_playgrounds["city"] = "Oulu"
oulu_playgrounds["has_supervised_activities"] = None
oulu_playgrounds["address"] = None

# names are all messed up
# 564 PORKKANAKUJAN LEIKKIPAIKKA -> Porkkanakujan leikkipaikka
# drop everything before first space
oulu_playgrounds["name"] = oulu_playgrounds["name"].str.split(" ", n=1).str[1]
# Capitalize first letter of name
oulu_playgrounds["name"] = oulu_playgrounds["name"].str.capitalize()

oulu_playgrounds["data_source"] = "municipality"
oulu_crs = oulu_playgrounds.geometry.crs.to_epsg() # should be epsg:3133

write_seed_file(oulu_playgrounds, oulu_seed_path, oulu_crs)

oulu_playgrounds.head()

,name,geometry,city,has_supervised_activities,address,data_source
0,Porkkanakujan leikkipaikka,POINT Z (471596.512 7203763.966 0),Oulu,None,None,municipality
1,Kuusikkotien leikkipaikka,POINT Z (471095.125 7204974.948 0),Oulu,None,None,municipality
2,Hovinsuon leikkipaikka,POINT Z (477317.479 7214402.921 0),Oulu,None,None,municipality
3,Hiukkavaaran keskusleikkipuisto,POINT Z (480838.235 7212632.045 0),Oulu,None,None,municipality
4,Peltolan leikkipuisto,POINT Z (477311.373 7212980.141 0),Oulu,None,None,municipality


# Rest of the country

For the rest of the country, we can use OSMnx to get the playgrounds.

In [50]:
osm_input_path = playgrounds_dir / "osm.gpkg"
osm_seed_path = seed_dir / "seed_osm.sql"
#osm_playgrounds = osmnx.features_from_place("Finland", tags={"leisure": "playground"})
#osm_playgrounds = osm_playgrounds[osm_playgrounds["name"].notna()]
#osm_playgrounds.to_file(osm_input_path, driver="GPKG")


In [51]:
munuicipalities_path = data_dir / "municipalities.gpkg"
municipalities = gpd.read_file(munuicipalities_path)
municipalities = municipalities[['nimi', 'geometry']]
municipalities = municipalities.rename(columns={'nimi': 'city'})

In [52]:
# generate address from add:street and add:housenumber
osm_playgrounds = gpd.read_file(osm_input_path)
osm_playgrounds = osm_playgrounds.sjoin(municipalities.to_crs(osm_playgrounds.crs), how="left", predicate="intersects")
osm_playgrounds["address"] = osm_playgrounds['addr:street'] + " " + osm_playgrounds['addr:housenumber'] + ", " + osm_playgrounds['city']

# set address to None if it is empty
osm_playgrounds.loc[osm_playgrounds['address'].isna(), 'address'] = None

columns = ['name', 'address', 'geometry', 'city']
osm_playgrounds = osm_playgrounds[columns]

# drop playgrounds that belong to cities that are already handled
osm_playgrounds = osm_playgrounds[~osm_playgrounds['city'].isin(cities_with_open_data)]

# set city to None if it is empty
osm_playgrounds.loc[osm_playgrounds['city'].isna(), 'city'] = None

# use ETRS-TM35FIN
osm_playgrounds = osm_playgrounds.to_crs(epsg=3067)

# convert polygon geometries to centroid points
osm_playgrounds["geometry"] = osm_playgrounds.geometry.centroid

osm_playgrounds["data_source"] = "osm"
osm_crs = osm_playgrounds.geometry.crs.to_epsg() # should be epsg:3067

write_seed_file(osm_playgrounds, osm_seed_path, osm_crs)

osm_playgrounds.head()

,name,address,geometry,city,data_source
0,Tarharannanpuisto,None,POINT (309244.299 7083170.535),Kokkola,osm
1,Laaksopuisto,None,POINT (310763.005 7082584.192),Kokkola,osm
2,Invalidikylänpuisto,None,POINT (308597.349 7084844.034),Kokkola,osm
6,Naisvuoren leikkipuisto,None,POINT (514447.371 6840070.253),Mikkeli,osm
7,Riskanpuisto,None,POINT (308533.5 7085102.46),Kokkola,osm


## Unnamed playgrounds

OSM has a lot of playgrounds that are not named. These are filtered out in the previous step. Here we get the rest of them.

In [53]:
osm_unnamed_input_path = playgrounds_dir / "osm_unnamed.gpkg"
osm_unnamed_seed_path = seed_dir / "seed_osm_unnamed.sql"
# osm_unnamed_playgrounds = osmnx.features_from_place("Finland", tags={"leisure": "playground"})
# osm_unnamed_playgrounds = osm_unnamed_playgrounds[osm_unnamed_playgrounds["name"].isna()]
# osm_unnamed_playgrounds.to_file(osm_unnamed_input_path, driver="GPKG")


In [54]:
# generate address from add:street and add:housenumber
osm_unnamed_playgrounds = gpd.read_file(osm_unnamed_input_path)
osm_unnamed_playgrounds = osm_unnamed_playgrounds.sjoin(municipalities.to_crs(osm_unnamed_playgrounds.crs), how="left", predicate="intersects")
osm_unnamed_playgrounds["address"] = osm_unnamed_playgrounds['addr:street'] + " " + osm_unnamed_playgrounds['addr:housenumber'] + ", " + osm_unnamed_playgrounds['city']

# set address to None if it is empty
osm_unnamed_playgrounds.loc[osm_unnamed_playgrounds['address'].isna(), 'address'] = None

columns = ['name', 'address', 'geometry', 'city']
osm_unnamed_playgrounds = osm_unnamed_playgrounds[columns]

# drop playgrounds that belong to cities that are already handled
osm_unnamed_playgrounds = osm_unnamed_playgrounds[~osm_unnamed_playgrounds['city'].isin(cities_with_open_data)]

# set city to None if it is empty
osm_unnamed_playgrounds.loc[osm_unnamed_playgrounds['city'].isna(), 'city'] = None

# use ETRS-TM35FIN
osm_unnamed_playgrounds = osm_unnamed_playgrounds.to_crs(epsg=3067)

# convert polygon geometries to centroid points
osm_unnamed_playgrounds["geometry"] = osm_unnamed_playgrounds.geometry.centroid

osm_unnamed_playgrounds["data_source"] = "osm"
osm_unnamed_crs = osm_unnamed_playgrounds.geometry.crs.to_epsg() # should be epsg:3067

write_seed_file(osm_unnamed_playgrounds, osm_unnamed_seed_path, osm_unnamed_crs)

osm_unnamed_playgrounds.head()

,name,address,geometry,city,data_source
0,None,None,POINT (310506.075 7084462.008),Kokkola,osm
2,None,None,POINT (308797.537 7083557.776),Kokkola,osm
3,None,None,POINT (701610.733 6954059.168),Ilomantsi,osm
9,None,None,POINT (349351.297 6663477.091),Siuntio,osm
11,None,None,POINT (594084.03 6786899.957),Imatra,osm
